In [ ]:
!pip install onnx
!pip install onnxsim

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import torch.utils as utils
from torchvision import datasets, transforms
from PIL import Image
from io import BytesIO
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
im2tensor = transforms.ToTensor()

def imfile2tensor(filename):
    img = Image.open(filename)
    alpha = Image.fromarray(np.zeros(img.size), mode='L')
    img.putalpha(alpha) # Unity RenderTextureの構造に一致させる
    tensor = im2tensor(img)
    return tensor

In [4]:
class UnityDataset(torch.utils.data.Dataset):
    def __init__(self, begin, end, transform = None):
        self.transform = transform
        # images
        image_title = '/content/drive/MyDrive/Sentis/Image'
        self.data = list()
        for i in range(begin, end):
            image_name = image_title + str(i).zfill(4) + '.png'
            tensor = imfile2tensor(image_name)
            self.data.append(tensor)
        # labels
        label_name = '/content/drive/MyDrive/Sentis/Visibility.csv'
        with open(label_name) as label_file:
            labels = label_file.readlines()
        self.label = []
        for i in range(begin, end):
            l = int(labels[i].split('\n')[0])
            self.label.append(torch.FloatTensor([l]))
        self.num_items = len(self.label)

    def __len__(self):
        return self.num_items

    def __getitem__(self, idx):
        out_data = self.data[idx]
        out_label = self.label[idx]
        if self.transform:
            out_data = self.transform(out_data)
        return out_data, out_label

In [8]:
# 1000個中900個を訓練データ
trainset = UnityDataset(0, 900)
# 残り100個をテストデータに
testset = UnityDataset(900, 1000)
dataloader = torch.utils.data.DataLoader(
    trainset,
    batch_size = 100,
    shuffle = True)

In [9]:
class Detector(nn.Module):
    def __init__(self, im_size, in_channels, num_hiddens):
        super(Detector, self).__init__()
        # プーリング層を介して2つの畳み込み層を設定
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=num_hiddens//2,
            kernel_size=3,
            stride=1,
            padding=1)
        self.pool1 = nn.MaxPool2d(
            kernel_size=3,
            stride=3)
        self.conv2 = nn.Conv2d(
            in_channels=num_hiddens//2,
            out_channels=num_hiddens,
            kernel_size=3,
            stride=1,
            padding=1)
        # 全結合層への入力次元数を計算
        nfeatures = np.floor((im_size - 1) / 3.0 + 1)   #conv1
        nfeatures = np.floor((nfeatures - 3) / 3.0 + 1) #pool1
        nfeatures = int(nfeatures) * 3
        nfeatures = num_hiddens * nfeatures * nfeatures
        # 全結合層2つを通じて出力
        self.linear1 = nn.Linear(nfeatures, nfeatures // 2)
        self.linear2 = nn.Linear(nfeatures // 2, 1)

    def forward(self, inputs):
        z = inputs[:, 0:3, :, :] # remove alpha channel
        z = self.conv1(z)
        z = F.relu(z)
        z = self.pool1(z)
        z = self.conv2(z)
        z = torch.flatten(z, 1)
        z = self.linear1(z)
        z = F.relu(z)
        z = self.linear2(z)
        return F.sigmoid(z) # nn.BCELossを用いるためsigmoidを利用

In [10]:
model = Detector(64, 3, 6).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1.0e-3)

In [ ]:
model.train()
num_epochs = 50
loss_list = []
for i in range(num_epochs):
    losses = []
    for x, label in dataloader:
        model.zero_grad()
        x = x.to(device)
        y = model(x)
        loss = criterion(y, label)
        loss.backward()
        optimizer.step()
        losses.append(loss.cpu().detach().numpy())
    loss_list.append(np.average(losses))
    print("EPOCH: {} loss: {}".format(i, np.average(losses)))

torch.save(model.state_dict(), 'sentis_weights.pth')

In [ ]:
# テスト

testloader = torch.utils.data.DataLoader(
    testset,
    batch_size = 100,
    shuffle = True)

def discretize(y):
    return 1.0 if y[0] >= 0.5 else 0.0

model.eval()
num_total = 0
num_oks = 0
with torch.no_grad():
    for x, t in testloader:
        x = x.to(device)
        y = model(x)
        for iy, it in zip(y, t):
            if discretize(iy) == it[0]:
                num_oks += 1
            num_total += 1
    print(num_oks / num_total)

In [ ]:
import onnx
import onnxsim

# 出力するモデルの設定
model = Detector(
    im_size=64,
    in_channels=3,
    num_hiddens=6).to(device)
checkpoint = torch.load('sentis_weights.pth')
model.load_state_dict(checkpoint)

# onnxファイルへの出力
onnx_file = 'sentis.onnx'
torch.onnx.export(
    model=model,
    args=torch.randn((1, 4, 64, 64)), # 入力値ダミー
    f='sentis.onnx', # モデル名
    opset_version=15, # Sentis1.3ではopsetバージョンは15に設定
    export_params=True, # 必ずTrue
    do_constant_folding=True, # True/Falseどちらでもよい（はず）
    input_names=['inputs'],  # 入力テンソルは1つ
    output_names=['output'], # 出力テンソルも1つ
)

# 型の推定
#model_onnx1 = onnx.load(onnx_file)
#model_onnx1 = onnx.shape_inference.infer_shapes(model_onnx1)
#onnx.save(model_onnx1, onnx_file)

# モデル構造の最適化
#model_onnx2 = onnx.load(onnx_file)
#model_simp, check = onnxsim.simplify(model_onnx2)
#onnx.save(model_simp, onnx_file)